In [ ]:
from pathlib import Path
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")
sns.set_context("talk", font_scale=1)
sns.set_palette("colorblind")

nvalloc = Path("write/220826-145416")
buddy = Path("write/220826-151355")


In [ ]:
nvalloc_d = pd.read_csv(nvalloc / "out.csv")
nvalloc_d["alloc"] = "NVAlloc"
buddy_d = pd.read_csv(buddy / "out.csv")
buddy_d["alloc"] = "Kernel"


pgd = pd.concat([nvalloc_d, buddy_d], ignore_index=True)
pgd["cores"] = pgd["x"]
pgd = pgd[["cores", "alloc", "max", "unmap", "iteration"]].melt(
    id_vars=["cores", "alloc", "iteration"],
    value_vars=["max", "unmap"], value_name="time")
# pgd = pgd.groupby(["cores", "alloc", "variable", "mem"]).min()
order = ["Kernel", "NVAlloc"]
g = sns.relplot(data=pgd, kind="line", x="cores", y="time",
                col="variable", style="alloc", hue="alloc", markers=True,
                style_order=order, hue_order=order)
g.set(xticks=[1, 16, 32, 48, 64])
g.set(ylim=(0, 30000))
g.legend.set_title("Allocator")
g.set(ylabel="time in ns")
g.set(xlabel="cores")
g.set_titles("{col_name}")
